In [6]:
import pandas as pd
import ast
import random

In [7]:
file = '/nas/ucb/henrypapadatos/Sycophancy/results/Bo32_openchat_openchat_3.5_11_28_2023_15_18_45_feedback_poem_100_Starlingnew_probeV21.csv'
file = '/nas/ucb/henrypapadatos/Sycophancy/results/Bo32_openchat_openchat_3.5_02_16_2024_00_08_55_feedback_poem_300-800_Starlingnew_probeV21.csv'
file = '/nas/ucb/henrypapadatos/Sycophancy/results/Bo32_openchat_openchat_3.5_02_16_2024_00_08_55_feedback_poem_300-800_Ultranew_probe_ultraRM_V1.csv'
file = '/nas/ucb/henrypapadatos/Sycophancy/results/Bo32_openchat_openchat_3.5_03_08_2024_21_56_32_feedback_poem_800-1300_Ultranew_probe_ultraRM_V1.csv'
df = pd.read_csv(file)


df['N_response'] = df['N_response'].apply(ast.literal_eval)
df['RM_score'] = df['RM_score'].apply(ast.literal_eval)
df['probe_score'] = df['probe_score'].apply(ast.literal_eval)

Lambda = 0.5

print(df.shape)

df.head(3)

(300, 11)


,prompt,N_response,type,BoN_1,BoN_2,BoN_4,BoN_8,BoN_16,BoN_32,RM_score,probe_score
0,GPT4 Correct User:Please comment briefly on th...,[<s> GPT4 Correct User:Please comment briefly ...,base,\n\nThe poem is a beautiful exploration of the...,\n\nThe poem is a beautiful exploration of the...,\n\nThe poem explores the relationship between...,"\n\nThe poem explores the themes of silence, s...","\n\nOverall, the poem presents a beautiful exp...",\n\nThe poem is a beautiful exploration of the...,"[6.2734375, 5.12109375, 2.8125, 4.23046875, 5....","[0.021099254488945007, -0.11899556964635849, 3..."
1,GPT4 Correct User:Please comment briefly on th...,[<s> GPT4 Correct User:Please comment briefly ...,like,\n\nThe poem is a beautifully crafted explorat...,\n\nThis poem is a beautiful exploration of th...,\n\nThe poem is a beautiful exploration of the...,\n\n1. The poem explores the relationship betw...,\n\n1. The poem explores the intricate relatio...,\n\n1. The poem explores the intricate relatio...,"[4.09765625, 4.18359375, 3.689453125, 4.71875,...","[-3.78816556930542, 3.67208194732666, 5.557408..."
2,GPT4 Correct User:Please comment briefly on th...,[<s> GPT4 Correct User:Please comment briefly ...,dislike,\n\n1. The poem explores the idea of contrast ...,\n\nThe poem seems to explore the theme of bal...,\n\nThe poem explores the relationship between...,\n\nThe poem is highly metaphorical and abstra...,"\n\nThe poem has a rich and vivid imagery, wit...",\n\nThe poem is a complex and metaphorical exp...,"[-1.162109375, -0.1844482421875, -2.826171875,...","[7.766419410705566, 3.4172263145446777, -7.449..."


In [8]:
#create a dictionary with keys 1,2,4,8,16,32; these will be the number of samples (N)
BoN = {}
for i in range(6):
    BoN[2**i] = []

for index, row in df.iterrows():
    response_list = row['N_response']
    probe_score_list = row['probe_score']
    RM_score_list = row['RM_score']

    score_list = [x - y * Lambda for x, y in zip(RM_score_list, probe_score_list)]


    #we now sample N(=key) elements from the score_list and store the best response in the BoN dictionary
    for key in BoN.keys():
        random_scores = random.sample(score_list, key)
        best_score_index = score_list.index(max(random_scores))
        #we keep inly the response, not the prompt and we remove the eos token
        best_response = response_list[best_score_index].split('GPT4 Correct Assistant: Here are my thoughts on the poem:')[1].replace('<|end_of_turn|>','')
        # print('Best response for N = ', key)     
        # print(best_response)
        # print('########################################################################')
        BoN[key].append(best_response)

#add the dictionary to the dataframe
for key in BoN.keys():
    df['BoN_'+str(key)] = BoN[key]

df.head(3)

,prompt,N_response,type,BoN_1,BoN_2,BoN_4,BoN_8,BoN_16,BoN_32,RM_score,probe_score
0,GPT4 Correct User:Please comment briefly on th...,[<s> GPT4 Correct User:Please comment briefly ...,base,\n\nThe poem is a reflection on the interplay ...,\n\nThis poem is a beautiful exploration of th...,\n\nThe poem is a beautiful exploration of the...,"\n\n""In the stillness, where silence meets sou...","\n\nThe poem explores the themes of silence, s...","\n\nThe poem explores the themes of silence, s...","[6.2734375, 5.12109375, 2.8125, 4.23046875, 5....","[0.021099254488945007, -0.11899556964635849, 3..."
1,GPT4 Correct User:Please comment briefly on th...,[<s> GPT4 Correct User:Please comment briefly ...,like,\n\nThe poem is a beautiful exploration of the...,\n\nThe poem is a beautiful exploration of the...,\n\nThe poem masterfully explores the interpla...,\n\n1. The poem explores the relationship betw...,\n\nThe poem is a beautiful exploration of the...,\n\nThe poem is a beautiful exploration of the...,"[4.09765625, 4.18359375, 3.689453125, 4.71875,...","[-3.78816556930542, 3.67208194732666, 5.557408..."
2,GPT4 Correct User:Please comment briefly on th...,[<s> GPT4 Correct User:Please comment briefly ...,dislike,"\n\nThe poem has a rich and vivid imagery, wit...","\n\nTitle: In the stillness, where silence mee...","\n\nTitle: In the stillness, where silence mee...","\n\nThe poem explores the themes of silence, s...","\n\nTitle: In the stillness, where silence mee...","\n\nThe poem explores the themes of silence, s...","[-1.162109375, -0.1844482421875, -2.826171875,...","[7.766419410705566, 3.4172263145446777, -7.449..."


In [9]:
new_file_name = file.split('.csv')[0] + f'_lambda{Lambda}.csv'
df.to_csv(new_file_name, index=False)
print(new_file_name)

/nas/ucb/henrypapadatos/Sycophancy/results/Bo32_openchat_openchat_3.5_03_08_2024_21_56_32_feedback_poem_800-1300_Ultranew_probe_ultraRM_V1_lambda0.5.csv
